# 3 Zenith-afhankelijkheid van een station.

Dit notebook sluit aan op het notebook 'HiSPARC_API'. Er wordt zowel informatie van het station als van de metingen verwerkt.

Een HiSPARC-station meet deeltjes in een deeltjeslawine die wordt veroorzaakt door een primair kosmisch deeltje. De deeltjes in de lawine bewegen globaal met de lichtsnelheid. De interactie van het primaire deeltje vindt plaats op een hoogte van tientallen kilometers. De doorsnede van de deeltjeslawine is slechts enkele hectometers. We kunnen we er dus van uitgaan dat deeltjes globaal in een vlak loodrecht op de snelheid van het primaire deeltje bewegen.

In [ ]:
import numpy as np
import sapphire
from sapphire import Station
detectors = Station(501).detectors()
for detector in detectors:
    print detector

De eigenschappen van een station zijn gedefinieerd met:

- 'mpv' : Most Probable Value. De spanning waarbij de grafiek van het aantal pulsen als functie van de spanning horizontaal is. Zie bijvoorbeeld: [http://data.hisparc.nl/show/stations/3401/2015/11/23/](http://data.hisparc.nl/show/stations/3401/2015/11/23/).
- 'alpha' : De hoek tussen de as gedefinieerd door de GPS-antenne en het midden van de scintillatorplaat en het Noorden. 
- 'beta' : De hoek tussen de lange zijde van de scintillatorplaat en het Noorden.
- 'radius' : De afstand van GPS- antenne tot het midden van de detector.

Een werkblad voor het maken van een stations-plattegrond is te vinden op:

[http://docs.hisparc.nl/infopakket/pdf/station_map.pdf](http://docs.hisparc.nl/infopakket/pdf/station_map.pdf)

De afstand tussen twee detectoren is te bepalen met de cosinusregel. Dit is in een functie te beschrijven:

In [ ]:
def afstand(detector_1, detector_2):
    '''
    
    '''
    c = detector_2['radius']
    b = detector_1['radius']
    alpha = np.radians(detector_2['alpha'] - detector_1['alpha'])
    # numpy werkt meet de hoek in radialen.
    return (b ** 2 + c ** 2 - 2 * b * c * np.cos(alpha)) ** .5

De afstand tussen detector 1 (telt als 0) en 4 (telt als 3) is nu te berekenen met:

In [ ]:
print afstand(detectors[0], detectors[3])

In het notebook python_data_retrieval is uitgelegd hoe data op te halen is:

In [ ]:
from sapphire import quick_download
data = quick_download(508)

In [ ]:
tijden = [10, 11, 12, 13] # Met deze waarden wordt naar de aankomsttijden bij de detectoren 1, 2, 3 en 4 gewezen.
for tijd in tijden:
    print data.root.s508.events[0][tijd]

In [ ]:
def zenithoeken(event, detectors):
    ''' 
    De zenithoek is de hoek tussen het golf-front en de horizon, 
    of ook tussen de as naar het zenith en de as van de deeltjeslawine.
    
    parameters
    event: een enkel event uit de opgehaalde data
    detectors: de detectorinformatie uit de API
    
    returns
    een array met de hoek tussen de as door twee detectors en het deeltjes front.
    '''
    c = 0.2997 # in m/ns
    zenith = []
    for i in range(0, 3):
        for j in range(i + 1, 4):
            schuine = afstand(detectors[i], detectors[j])
            overstaande = c * (event[i + 10] - event[j + 10]) / 100
            try:
                angle = np.degrees(np.arcsin(overstaande / schuine))
            except:
                angle = -999
            zenith.append(angle)
    return zenith

In [ ]:
event = data.root.s508.events[0]
print zenithoeken(event, detectors)

In [ ]:
selecties = range(0, 10)
for selectie in selecties:
    event = data.root.s508.events[selectie]
    print zenithoeken(event, detectors)

Hierboven zijn de gereconstrueerde hoeken voor de eerste 10 events voor alle combinaties van 2 detectoren in een set van 4 detectoren berekend. De hoeken zijn gedefinieerd als de hoek tussen de as door de detector en het hart van de deeltjeslawine.

Een beschrijving van de zenith-hoek is te vinden op: [http://docs.hisparc.nl/infopakket/pdf/richting_reconstructie.pdf](http://docs.hisparc.nl/infopakket/pdf/richting_reconstructie.pdf).